# Base Data Loader

## Introduction

Add a Markdown cell at the beginning of the notebook explaining the overall objective, including the use of LOAD CSV for importing data into Neo4j, and any prerequisites such as CSV file locations and database setup.

## Setup and Configuration

### local neo4j

Run the neo4j container with the data and import mount-points something like this

``` bash
docker run \
    --restart always \
    --publish=7474:7474 --publish=7687:7687 \
    --env NEO4J_AUTH=neo4j/password \
    --volume=/Volumes/DevStorage/Databases/neo4j/neo4jdata:/data \
    --volume=/Volumes/DevStorage/github/rlhatcher/ronaldhatcherblog/data/data_clean:/import \
    neo4j:5.16.0
```

The directory containing this notebook and the CSV data files should map to the import folder in the neo4j image.

### imports

In [ ]:
from neo4j import GraphDatabase

### database connection

In [ ]:
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))

## Queries

In [ ]:
queries = {
    "clear_graph": "MATCH (n) DETACH DELETE n",
    "count_graph": "MATCH (n) RETURN COUNT(n)",
    "load_person": """
        CREATE (:Person {
            email: 'ronaldhatcher@mac.com',
            family_name: 'Hatcher',
            given_name: 'Ronald',
            id: 'kp_878e41a7d93846cca23114ad549e8126',
            picture: 'https://lh3.googleusercontent.com/a/ACg8ocKicc19o8E_h-mOGCLngSlTsFxJLcIGONz_ntc4RW5_3Q=s96-c'})
        """,
    "load_mfg": """
        LOAD CSV WITH HEADERS FROM $url AS line
        CREATE (:Manufacturer {name: line.name, mfgID: line.mfgID})
        """,
    "load_motor": """
        LOAD CSV WITH HEADERS FROM $url AS line
        CREATE (:Motor {
          commonName: line.commonName,
          delays: line.delays,
          diameter: toFloat(line.diameter),
          infoUrl: line.infoUrl,
          totImpulseNs: toFloat(line.totImpulseNs),
          manufacturer: line.manufacturer,
          burnTimeS: toFloat(line.burnTimeS),
          propInfo: line.propInfo,
          length: toFloat(line.length),
          avgThrustN: toFloat(line.avgThrustN),
          dataFiles: line.dataFiles,
          impulseClass: line.impulseClass,
          sparky: line.sparky,
          caseInfo: line.caseInfo,
          propWeightG: toFloat(line.propWeightG),
          certOrg: line.certOrg,
          motorId: line.motorId,
          availability: line.availability,
          maxThrustN: toFloat(line.maxThrustN),
          totalWeightG: toFloat(line.totalWeightG),
          designation: line.designation,
          updatedOn: line.updatedOn,
          type: line.type,
          mfgID: line.mfgID
        })
        """,
    "load_kit": """
        LOAD CSV WITH HEADERS FROM $url AS row
        MERGE (kit:Kit {uniqueID: row.UniqueID})
        ON CREATE SET
          kit.name = row.Name,
          kit.url = row.url,
          kit.imageSrc = row['image_src'],
          kit.recommendedEngines = row['Recommended Engines'],
          kit.projectedMaxAltitude = row['Projected Max Altitude'],
          kit.recoverySystem = row['Recovery System'],
          kit.length = toFloat(row.Length),
          kit.diameter = toFloat(row.Diameter),
          kit.estimatedWeight = toFloat(row['Estimated Weight']),
          kit.estimatedAssemblyTime = row['Estimated Assembly Time'],
          kit.finMaterials = row['Fin Materials'],
          kit.decalType = row['Decal Type'],
          kit.launchSystem = row['Launch System'],
          kit.launchRodSize = row['Launch Rod Size'],
          kit.instructions = row.instructions,
          kit.ageRecommendation = row['Age Recommendation'],
          kit.mfgID = row.mfgID,
          kit.height = toFloat(row.Height),
          kit.weight = toFloat(row.Weight),
          kit.motorMount = row['Motor Mount'],
          kit.parachuteSize = row['Parachute Size'],
          kit.shockCordType = row['Shock Cord Type'],
          kit.shockCordMount = row['Shock Cord Mount'],
          kit.finThickness = toFloat(row['Fin Thickness']),
          kit.ringThickness = toFloat(row['Ring Thickness']),
          kit.instructionsUrl = row.Instructions,
          kit.price = toFloat(row.Price),
          kit.currency = row.Currency,
          kit.sku = row.SKU,
          kit.stockStatus = row['Stock Status'],
          kit.description = row.Description,
          kit.links = row.Links,
          kit.parachute = row.Parachute,
          kit.finArray = row['Fin Array'],
          // Using CASE to conditionally set the complexity property
          kit.complexity = CASE WHEN row.Complexity IS NOT NULL THEN row.Complexity ELSE NULL END;
        """,
    "load_rocket": """
        LOAD CSV WITH HEADERS FROM $url AS line
        CREATE (:Rocket {id: line.id, name: line.name, mfgID: line.mfgID, image: line.image})
        """,
    "load_model": """
        LOAD CSV WITH HEADERS FROM $url AS line
        CREATE (:Model:Rocket {name: line.name, id: line.id, image: line.image})
        """,
    "load_design": """
        LOAD CSV WITH HEADERS FROM $url AS line
        CREATE (:Design 
          { id: line.id, 
            rocket: line.rocket, 
            name: line.name, 
            filename: line.filename, 
            fileURL: line.fileURL,
            stages: toInteger(line.stages),
            massEmpty: toFloat(line.massEmpty),
            stabilityCal: toFloat(line.stabilityCal),
            stabilityPct: toFloat(line.stabilityPct),
            cg: toFloat(line.cg),
            cp: toFloat(line.cp),
            length: toFloat(line.length),
            maxDiameter: toFloat(line.maxDiameter)
          })
        """,
    "mfg_makes_motor":  """
        MATCH (manufacturer:Manufacturer), (motor:Motor) 
        WHERE motor.mfgID = manufacturer.mfgID 
        MERGE (manufacturer)-[:MAKES]->(motor);
        """,
    "mfg_makes_kit": """
        MATCH (manufacturer:Manufacturer), (kit:Kit)
        WHERE kit.mfgID = manufacturer.mfgID 
        MERGE (manufacturer)-[:MAKES]->(kit);
        """,
    "mfg_makes_rocket": """
        MATCH (manufacturer:Manufacturer), (rocket:Rocket)
        WHERE rocket.mfgID = manufacturer.mfgID 
        MERGE (manufacturer)-[:MAKES]->(rocket);
        """,
    "vorb_lone": """
        MATCH (r:Rocket {mfgID: 'vorb', id: 'launcherone'}), (m:Model {id: 'lone'})
        MERGE (m)-[:BASED_ON]->(r)
        """,
    "hughes_aim54": """
        MATCH (r:Rocket {mfgID: 'hughes', id: 'aim54a'}), (m:Model {id: 'phx'})
        MERGE (m)-[:BASED_ON]->(r)
        """,
    "aim54_kit": """
        MATCH (k:Kit {uniqueID: '9e295916-5309-4471-847d-4d42c814a806'}), (m:Model {id: 'phx'})
        MERGE (m)-[:BASED_ON]->(k)
        """,
    "person_models": """
        MATCH (p:Person {id: "kp_878e41a7d93846cca23114ad549e8126"}), (m:Model)
        WHERE m.id IN ['lone', 'phx', 'bfhr']
        MERGE (p)-[:OWNS]->(m)
        """
}

## Query Execution

In [ ]:
def execute_query(driver, query_key, parameters=None):
    with driver.session() as session:
        result = session.execute_write(lambda tx: tx.run(queries[query_key], parameters))
        return result

## Data Loading

In [ ]:
execute_query(driver, "clear_graph")

# Nodes
execute_query(driver, "load_mfg", {"url": "file:///manufacturers.csv"})
execute_query(driver, "load_motor", {"url": "file:///motors.csv"})
execute_query(driver, "load_kit", {"url": "file:///kits.csv"})
execute_query(driver, "load_rocket", {"url": "file:///rockets.csv"})
execute_query(driver, "load_person")
execute_query(driver, "load_model", {"url": "file:///models.csv"})
execute_query(driver, "load_design", {"url": "file:///designs.csv"})

# Relationships
execute_query(driver, "mfg_makes_motor")
execute_query(driver, "mfg_makes_kit")
execute_query(driver, "mfg_makes_rocket")

# Hacks
execute_query(driver, "vorb_lone")
execute_query(driver, "hughes_aim54")
execute_query(driver, "aim54_kit")
execute_query(driver, "person_models")

## Validation

After loading the data, include steps to validate the import process (e.g., count nodes and relationships).

```python
# Example validation query
result = execute_query(driver, "count_people")  # Assume you have defined this query
for record in result:
    print("Number of people:", record["count"])
```

Add a concluding Markdown cell summarizing the work done and any observations or next steps.

## Cleanup

In [ ]:
driver.close()